# Heston model: Analytical solution

In this notebook, we show how to price European options with the Heston stochastic volatility model.

The Heston model is defined by the following system of stochastic differential equations where the stock price follows a geometric Brownian motion and its variance follows a Cox-Ingersoll-Ross (CIR) process,

\begin{align}
	&dS_t = \mu S_t dt + \sqrt{v_t} S_t dW_t^{(1)} \\
	&dv_t = \kappa ( \theta - v_t ) dt + \sigma \sqrt{v_t} dW_t^{(2)} \\
	&dW_t^{(1)} dW_t^{(2)} = \rho dt
\end{align}

where $S_t$ is the underlying stock price, $v_t$ is its variance, $\mu$ is the expected return, $\kappa$ is the mean-reversion rate, $\theta$ is the long-term variance, $\sigma$ is the volatility of volatility, $W_t$ is a Wiener process and $\rho$ is the correlation coefficient between the Brownian motions. There are also two initial parameters, $S_0$ and $v_0$ which are non-negative.

The process $v_t$ is strictly positive if the Feller condition is satisfied. It is given by

\begin{equation}
    2 \kappa \theta > \sigma^2.
\end{equation}

The Heston model yields a semi-closed form solution for pricing European equity options. The price for a European call option, $C(S, v, t)$, at time $t$ with time to maturity $\tau = T - t$ and strike $K$ is given by

\begin{equation}
	C(S, v, t) = S_t e^{-q \tau} P_1 - K e^{-r \tau} P_2 
\end{equation}

where $q$ is the dividend rate and $r$ is the interest rate and $P_j, j = \{1, 2\}$ are risk-neutral probabilities obtained by inverting the characteristic function $f_j$ (given below). Hence,

\begin{equation}
    P_j = \frac{1}{2} + \frac{1}{\pi} \int_0^{\infty} \operatorname{Re} \left[ \frac{e^{-i \phi \ln(K)}f_j}{i \phi} \right] d\phi
\end{equation}

where

\begin{align}
	&f_j = \exp(C_j + D_j v_t + i \phi \ln S_t) \\
	&C_j = (r - q) \phi i \tau + \frac{\kappa \theta}{\sigma^2} \left[ (b_j - \rho \sigma \phi i + d_j) \tau - 2 \ln \left( \frac{1 - g_j e^{d_j \tau}}{1 - g_j} 	\right) \right] \\[5pt]
	&D_j = \frac{b_j - \rho \sigma \phi i + d_j}{\sigma^2} \left( \frac{1 - e^{d_j \tau}}{1 - g_j e^{d_j \tau}} \right) \\
	&g_j = \frac{b_j - \rho \sigma \phi i + d_j}{b_j - \rho \sigma \phi i - d_j} \\
	&d_j = \sqrt{(\rho \sigma \phi i)^2 - \sigma^2 ( 2 u_j \phi i - \phi^2 )}
\end{align}

In the above expressions, $i = \sqrt{-1}$ is the imaginary unit, $u_1 = \frac{1}{2}$, $u_2 = - \frac{1}{2}$, $b_1 = \kappa + \lambda - \rho \sigma$ and $b_2 = \kappa + \lambda$. The parameter $\lambda$ is the market price of volatility risk. 

First, let's implement the function to price call options. Note that we price the option at time $t = 0$, therefore, $\tau = T$.

In [1]:
import numpy as np
from scipy.integrate import quad

# Heston call price
def Heston_call_price(S0, v0, K, T, r, q, kappa, theta, sigma, rho, lmbda):
    p1 = p_Heston(S0, v0, K, r, q, T, kappa, theta, sigma, rho, lmbda, 1)
    p2 = p_Heston(S0, v0, K, r, q, T, kappa, theta, sigma, rho, lmbda, 2)
    return S0 * np.exp(-q*T) * p1 - K * np.exp(-r*T) * p2

# Heston probability
def p_Heston(S0, v0, K, r, q, T, kappa, theta, sigma, rho, lmbda, j):
    integrand = lambda phi: np.real(np.exp(-1j * phi * np.log(K)) \
                                    * f_Heston(phi, S0, v0, T, r, q, kappa, theta, sigma, rho, lmbda, j) \
                                    / (1j * phi))    
    integral = quad(integrand, 0, 100)[0]
    return 0.5 + (1 / np.pi) * integral

# Heston characteristic function
def f_Heston(phi, S0, v0, T, r, q, kappa, theta, sigma, rho, lmbda, j):
        
    if j == 1:
        u = 0.5
        b = kappa + lmbda - rho * sigma
    else:
        u = -0.5
        b = kappa + lmbda
    
    a = kappa * theta
    d = np.sqrt((rho * sigma * phi * 1j - b)**2 - sigma**2 * (2 * u * phi * 1j - phi**2))
    g = (b - rho * sigma * phi * 1j + d) / (b - rho * sigma * phi * 1j - d)
    C = (r - q) * phi * 1j * T + (a / sigma**2) \
            * ((b - rho * sigma * phi * 1j + d) * T - 2 * np.log((1 - g * np.exp(d * T))/(1 - g)))
    D = (b - rho * sigma * phi * 1j + d) / sigma**2 * ((1 - np.exp(d * T)) / (1 - g * np.exp(d * T)))
    
    return np.exp(C + D * v0 + 1j * phi * np.log(S0))

Now, let's define some parameters and price a call option. We also price the corresponding put using the put-call parity, i.e.,

\begin{equation}
    V_p = V_c + Ke^{-rT} - S_0e^{-qT}
\end{equation}

where $V_p$ is the price of the put option and $V_c$ is the price of the call option.

In [2]:
# Parameters
T = 1        # maturity
S0 = 55      # spot price
K = 50       # strike price
r = 0.04     # risk-free interest rate
q = 0.02     # dividend rate
v0 = 0.04    # initial variance
rho = -0.7   # correlation between Brownian motions
kappa = 2    # mean reversion rate
theta = 0.04 # Long term mean of variance
sigma = 0.3  # volatility of volatility
lmbda = 0    # market price of volatility risk

# Option values
Vc = Heston_call_price(S0, v0, K, T, r, q, kappa, theta, sigma, rho, lmbda) # call
Vp = Vc + K*np.exp(-r*T) - S0*np.exp(-q*T) # put with put-call parity

print('Call price: ' + str(round(Vc, 5)))
print('Put price:  ' + str(round(Vp, 5)))

Call price: 7.78614
Put price:  1.91469
